In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import trino
import os
import sys
import pickle
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from dotenv import load_dotenv
from time import sleep

In [ ]:
pd.set_option("display.max_columns", None)

load_dotenv()

True

In [3]:
presto_user = os.getenv("PRESTO_USER")
presto_password = os.getenv("PRESTO_PASSWORD")

def get_df_from_qry(qry: str, verbose: bool = True) -> pd.DataFrame:
    username, password = os.environ["PRESTO_USER"], os.environ["PRESTO_PASSWORD"]

    conn = trino.dbapi.connect(
        host="presto-secure.data-infra.shopee.io",
        port=443,
        user=username,
        catalog="hive",
        http_scheme="https",
        source=f"(50)-(vnbi-dev)-({username})-(jdbc)-({username})-(SG)",
        auth=trino.auth.BasicAuthentication(username, password),
    )
    cur = conn.cursor()

    if verbose:
        thread = ThreadPoolExecutor(1)
        async_result = thread.submit(cur.execute, qry)

        bar_queue = tqdm()
        while not async_result.done():
            memory = cur.stats.get("peakmemorybytes", 0) * 10**-9
            perc = 0
            stt = cur.stats.get("state", "")
            if stt == "running":
                perc = round(
                    (cur.stats.get("completedsplits", 0) * 100.0)
                    / (cur.stats.get("totalsplits", 0)),
                    2,
                )
            status = f"{stt} {perc}% - memory {memory:,.0f}gb"
            bar_queue.set_description(status)
            bar_queue.update(1)
            sleep(5)
        bar_queue.close()
    else:
        cur.execute(qry)

    output = cur.fetchall()
    cols = [col.name for col in cur.description]

    return pd.DataFrame(output, columns=cols)

In [13]:
qry_prop_match = """
WITH march_data AS (
    SELECT 
        model_id, 
        item_cluster,
        seller_type,
        is_new_join,
        shop_id,
        impact_fee_net_take_rate AS take_rate_march,
        gmv AS gmv_march,
        cogs AS cogs_march,
        cogs_group AS cogs_group_march,
        total_orders AS total_orders_march,
        model_clicks AS model_clicks_march,
        model_impressions AS model_impressions_march,
        model_ads_clicks AS ads_clicks_march,
        model_ads_impressions AS ads_impressions_march,
        model_ads_expenditure_usd AS ads_spend_march,
        level1_global_be_category AS level1_global_be_category,
        level2_global_be_category AS level2_global_be_category,
        model_create_month AS model_create_month,
        active_months AS active_months
    FROM dev_vnbi_bd.price_elasticity_model_level
    WHERE grass_month = DATE '2025-03-01' 
    AND cogs IS NOT NULL 
    AND cogs != 0 
),
april_data AS (
    SELECT 
        model_id, 
        seller_type,
        is_new_join,
        shop_id,
        impact_fee_net_take_rate AS take_rate_april,
        gmv AS gmv_april,
        cogs AS cogs_april,
        cogs_group AS cogs_group_april,
        total_orders AS total_orders_april,
        model_clicks AS model_clicks_april,
        model_impressions AS model_impressions_april,
        model_ads_clicks AS ads_clicks_april,
        model_ads_impressions AS ads_impressions_april,
        model_ads_expenditure_usd AS ads_spend_april
    FROM dev_vnbi_bd.price_elasticity_model_level
    WHERE grass_month = DATE '2025-04-01' 
    AND cogs IS NOT NULL
    AND impact_fee_net_take_rate != 0 
    AND cogs!= 0 
)
SELECT 
    a.model_id,
    a.item_cluster,
    a.seller_type,
    a.is_new_join,
    a.shop_id,
    a.take_rate_march, b.take_rate_april,
    a.gmv_march, b.gmv_april,
    a.cogs_march, b.cogs_april,
    a.cogs_group_march, b.cogs_group_april,
    a.total_orders_march, b.total_orders_april,
    a.model_clicks_march, b.model_clicks_april,
    a.model_impressions_march, b.model_impressions_april,
    a.ads_clicks_march, b.ads_clicks_april,
    a.ads_impressions_march, b.ads_impressions_april,
    a.ads_spend_march, b.ads_spend_april,
    a.level1_global_be_category, a.level2_global_be_category, 
    a.model_create_month, a.active_months
FROM march_data a
JOIN april_data b 
    ON a.model_id = b.model_id
"""

In [8]:
qry_features = """
WITH filtered_model AS (
SELECT item_id, model_id
FROM mp_item.dim_model__vn_s0_live
WHERE grass_date = DATE '2025-07-16' - INTERVAL '2' DAY
)
SELECT 
f.model_id, c.level4_local_category
FROM filtered_model f
JOIN dev_vnbi_mkt.ds_mkp__item_lv4_category c
ON f.item_id = c.item_id
"""

In [ ]:
qry_ids = """
SELECT model_id, item_id
FROM mp_item.dim_model__vn_s0_live
WHERE grass_date = DATE '2025-07-22' - INTERVAL '2' DAY
    AND model_id IN (
        SELECT model_id
        FROM dev_vnbi_bd.price_elasticity_model_level
)
"""

In [ ]:
def download_qry_as_parquet(qry, file_name):
    temp = get_df_from_qry(qry)
    temp.to_parquet(file_name)

download_qry_as_parquet(qry_features, "features_list_23_07.parquet")